# Satisfiability of Systems of Polynomial (In)Equations in Prime Field

Given a system of polynomial statements, like the following:
$$
\begin{align*}
&4x^3 + 2x^2y^2 + 12x - y = 5 \\
&y^3 - 3x^2 \neq 7xy^3 +1 \\
&y \neq x,
\end{align*}
$$
the goal is to find if there is an evaluation of the variables ($x$ and $y$ in this case) in a field
mod some prime $p$ where all the statements are true. This can be eventually integrated into a SMT
solver, like Z3, and be used to prove statements about zkSNARKs circuits, as generated by ZoKrates.

It seems that inequations of the kind $x^2~\le~y$ does not really make sense in a mod $p$ field,
because the values wrap around, so a total ordering among the elements is not really meaningfully 
defined for most applications of the satisfiability solver. Alternatively, ranges of the kind
$a~\le~x^2~-~y~\lt~b$ looks much more useful, but I am not sure they will be in the scope of this
work, because I do not know if they would be useful for the goal of proving statements about zkSNARKs
circuits, so this work will be restricted to $=$ and $\neq$ constraints.

## Turning into a system of equations

The problem above can be reduced to the problem of finding the zeros of a set of polynomials,
i.e. the value of $\mathbf{x}$ such that $f_1(\mathbf{x})~=~f_2(\mathbf{x})~=~...~=~f_m(\mathbf{x})~=~0$,
where $\mathbf{x}$ is the vector $(x_1, x_2, ..., x_n)$. For the equalities is easy, just rearrange all terms to
the left-hand side, leaving 0 on the right-hand side. The equality in the example above will became
the polynomial:
$$
f_1 = 4x^3 + 2x^2y^2 + 12x - y - 5
$$

For the differences, we can use the "Rabinowitsch trick" (originally used in a proof of Hilbert's
Nullstellensatz), to turn all the differences into a single equation. We first isolate all terms
on the left-hand side, introduce a new variable $z$, and produce the following equation:
$$
zg_1g_2...g_k - 1 = 0
$$
where $g_i$ are the $\neq 0$ polynomials. This works because the equation only have a solution if
$z$ is the inverse of $g_1g_2...g_k$, and zero has no inverse. In other words, if any of the $g_i$
is zero, the term $g_1g_2...g_k$ vanishes and the remaining equation $-1=0$ has no solutions.

In the above example, the inequations will produce the following polynomial:
$$
f_2 = z (y^3 - 3x^2 - 7xy^3 - 1)(y - x) - 1
$$

The problem is then reduced to the problem of finding the common roots for a set of polynomials (that
belongs in the prime field, obviously, because roots of polynomials may not be defined in the same set
as the coefficients, for instance $x^2~-~2~=~0$ has no solution in integers and $x^2~+~2~=~0$ has no
solution in reals) and exclude all solutions that does not match the variables restrictions.

### Optimization idea

It might prove beneficial to use the square-free component of the polynomials (and also of $g_1g_2...g_k$.
in case of the polynomial from the inequalities), and not the of the original polynomials. This is
because square-free decomposition is cheap, may reduce the degree and size of the polynomial, simplifying
the solving procedure, and the solution is identical (except maybe for the introduced variable $z$, which
is not important to the problem).

## Solving a system of polynomial equations

Since we reduced our problem to a system of polynomial equations, we now have to solve it. I will now
describe the alternatives I am considering to solve it, and where I stand. I will assume the degrees of
the input polynomials are small (not much larger than 10) and the prime field is large (around $2^{254}$).

But before we try any expensive solving algorithm, lets get out of the way some special cases
that are easily handled. It is convenient to first try to reduce the set of polynomials
against itself (which is a step of any Gröbner Basis algorithm, see below), so that the system
is sanitized (all trivial zeros and obviously equivalent polynomials are removed), and some easy to
spot relations might be exposed, thus reducing the degree of some polynomials.

Special case #1: if the set of polynomials is empty, we consider it vacuously
satisfiable and finish. I don't see a good justification for an empty set to be considered
unsatisfiable, on the contrary: trivial zero polynomials are excluded during self-reduction, as they
don't affect the ideal in any way, which leaves the empty set in case the system is the single
polynomial $0$, which is an obviously and trivially solvable by any possible variable evaluation.

Special case #2: non-zero constant polynomial case: if the self-reduction returns a set
with a non-zero constant polynomial (and in this case, reduction definition guarantees it will be
the only polynomial left), it obviously can not be zero, so the system is unsatisfiable. In the
current implementation of reduction, if a non-zero constant polynomial is ever encountered, the
algorithm is cut short and returns immediately so we don't waste more time.

Then we iterate through all the polynomials and count 3 things: the maximum degree of the system $d$,
the number of variables $n$, and the total number of zero degree terms $t_z$.

Special case #3: if $t_z = 0$, there are no zero degree terms, then $(0,0,...,0)$ is a solution of the
system, so we can stop.

Special case #4: if $d = 1$ (can't be 0 because that was checked in special case #1 and #2) then system
is linear. Reduction is the generalization of a Gaussian elimination, so whatever remained of the system
is necessarily solvable. We can stop, and the system is already in triangular form, so it is trivial
to compute the unique solution.

Special case #5: if $n = 1$, reduction guarantees there will be only one polynomial left, since it is
monic, we can extract some root by using Cantor-Zassenhaus probabilistic algorithm (not implemented yet).

Special case #6: not implemented and not sure if it is worth, but in case of a single polynomial with
$n > 1$, [1] suggests factoring into irreducible factors: if any of the factors is absolutely
irreducible (there is a test for that [3]) then we consider it satisfiable and we are done (there
is a theorem that guarantees plenty of solutions if $d$ and $n$ are small enough compared to
the field size [1, 2, 5]). Each factor $f$ can then be handled independently like
the following (even less sure if it is worth): recursively test if the system $\{f = df/dx = 0\}$ is
satisfiable for some variable $x$ where $df/dx_i$ does not vanishes. The problem is satisfiable iff at
least one of these subsystems spawned by the factors is satisfiable.

I ran out of ideas for special cases, so now I will discuss the alternatives being explored for the
general case.

## Fast Algebraic Geometry Method

If the set of polynomials contains some very particular properties, namely, defines a prime ideal
(an ideal that is both primary and radical), and defines an absolutely irreducible algebraic set,
then algorithm [2] allows for finding one common root in probabilistic polynomial time.

The problem is, of course, this method is incomplete as not all problems will obey these properties.
It is unclear how many of practical problems would so that the algorithm is useful on its own.
I don't know either what happens if the algorithm is executed on a polynomial set without those
properties (will it fail? will it run forever? remains to be clarified). Also don't know how to
test if an ideal is radical without a full Gröbner basis, and don't know at all how to test if a
variety is absolutely irreducible.

Nothing on this front has been implemented.

## Robust Algebraic Geometry Method

There is another class of algorithms based on algebraic geometry (original probabilistic version
given in [1], deterministic version given in [7]). Compared to the Fast Algebraic Geometry Method,
instead of expecting the problem to be given as an absolutely irreducible variety, it recursively
decomposes the problem until it either eventually reach an absolutely irreducible variety (then
deciding for satisfiable) or it is no longer possible to decompose, deciding for unsatisfiable.

This algorithm is double exponential on the number of variables, and only works if $p > d^{n^{O(n)}}$,
where $p$ is the order of the prime field, $d$ is the maximum degree of the polynomials and $n$ is the
number of variable. Using our assumptions of $p \approx 2^{254}$ and $d \approx 10$, this algorithm
doesn't work in general for much more than a couple of variables.

It is unclear for how many instances of the problem this algorithm might work in practice, but at
least for the probabilistic version, this algorithm doesn't look practical at all.

Nothing of the variety decomposition algorithm has been implemented. From here, I only implemented
a test that tells if a single multivariate polynomial that happens to be absolutely irreducible has
any roots (what I called the Schmidt test, originally from [5]).

## Methods based on Gröbner Basis

Finding the reduced Gröbner basis of the polynomial set is often the first step for solving the system.
The best know algorithms for finding Gröbner basis are Faugère's F4 and F5 algorithms, and they have
exponential space complexity on the worst case. The Gröbner basis is defined for
a total order among the monomials (i.e. the variable product power parts like $x^3y^2$), which implies
a total order among the variables themselves. Each ordering has some useful properties: reverse
degree lexicographical order (degrevlex), also called graded reverse lexicographic order (grevlex)
is the fastest to calculate (exponential instead of double exponential). Lexicographical ordering
leaves the system in triangular format suitable for variable elimination.

Right now all I have implemented of this method is a quite bad version of Buchberger's algorithm,
which is much worse in practice than Faugère's F4 and F5 (but have the same worst case complexity),
which for a very small system is capable of computing the Gröbner basis in grevlex ordering, but I
don't believe it would terminate in a human lifetime for lexicographical ordering.

### Triangular System

For this approach, we need the Gröbner basis in lexicographical order, so that the system is in a
triangular format where we can eliminate one variable at a time. But Gröbner basis is much faster to
calculate on grevlex, so much that in practice lexicographical order is obtained by transforming from
grevlex via Gröbner Walk or some algorithm of the sort. According to [4], this order conversion
step is the slowest in this approach.

This will yield a new set of polynomials with the same roots, but in a triangular format, i.e. that
once the free variables are assigned arbitrarily, the system can be solved one variable a time, so
that the first polynomial has only one variable to be solved, which will leave the next polynomial
with only one variable after replacing the variables already known, and so forth until the system is
completely solved. 

For instance, in a 3 variable systems, if we define $x > y > z > 1$ in our ordering (1 must always be
the smallest), then the first variable that can be solved is $z$, then $y$, then $x$, assuming the
system is fully determined. If the system is not fully determined, and there are free variables, then
the lowest variables are the ones to be assigned.

Having an algorithm to find the roots one variable at a time, we have a branching problem, where each
variable assignment will lead to a new set of possible assignments for the next variable, and so on,
so that the naïve search expands into a typical NP problem.

Finding roots of a univariate polynomial in a prime field is relatively cheap, with the best known
algorithm being factorizing the polynomial into unique factors $(x - a_1)(x - a_2)...(x - a_n)$ where
$a_i$ are the roots of the polynomial. The most efficient algorithm for factorization in a prime
field is called Cantor-Zassenhaus algorithm, with complexity $O(d^3 \log p)$ where $d$ is the degree
of the polynomial and $p$ is the prime field's prime.

Nothing besides the bad Gröbner basis in grevlex is implemented for this approach, being Gröbner Walk (or
some alternative) and Cantor-Zassenhaus the most notable absences.

The unsolved problems of this approach includes how to choose the variable ordering optimally and
how the search for a variable assignment in the triangular system that is better than exhaustive
trial and error. It is unclear how to perform a local search in the triangular system, much less
how to prove unsatisfiability. Maybe [4] can shed a light into this.

### Primary Decomposition Method

Given polynomials in the ring $\mathbb{F}[x_1,x_2,...,x_n]$, every polynomial set defines an algebraic set,
which is the set of points in $\mathbb{F}^n$ where all polynomials evaluates to zero. This is the
set from where we seek solutions: by finding one point in the algebraic set, we have proven the problem to
be satisfiable, and by proving the algebraic set to be empty, we have proven the problem to be unsatisfiable.

The set of polynomials we are trying to solve defines an ideal, and by Lasker-Noether theorem we know that
every polynomial ideal is the intersection of a finite set of primary ideals (called the primary
decomposition of the ideal), which implies any algebraic set is a union of irreducible algebraic
sets, named varieties.

So the idea of this approach is to find the radical of the ideal defined by our polynomial set,
calculate its primary decomposition, and then, for each prime ideal found (which is primary and radical),
try to find a zero in its algebraic set. This seems to go back to the initial problem: we have a set of
polynomials for which we must find a common zero point. Well, not quite, because since we know we are
dealing with an algebraic variety, we may be able to handle it with algebraic geometry methods such as the
one described in [2].

This can also be the first approach of a hybrid method, where the bigger problem is reduced to smaller
sub-problems that can then be handled in parallel. Each subproblem we try to solve first by the Fast
Algebraic Geometry method, and if it fails, we can apply the triangular system method, local search
method, CDCL approach or something else.

Computing the Gröbner Basis is the first step of the primary decomposition algorithms and radical ideal
algorithms I saw (such as [6]).

Nothing of the sort have been implemented (besides Gröbner Basis itself).

The biggest issue of this method is that it is not complete: it will decide for satisfiability if it is
easy to extract a rational point from one of the varieties (i.e. the variety is absolutely irreducible)
and it will decide for unsatisfiability if the algebraic variety is empty on the extension field (Gröbner
basis turns out to be a single non-zero constant), but besides partitioning the problem, it won't help in
the case all varieties are reducible in the extension field.

### Base field restriction

The satisfiability problem can be solved by finding the Gröbner basis of the system extended with the
equations
$$
x^p - x = 0
$$
for each variable $x$ in the system (as per Fermat's little theorem, $x^p = x \mod p$). This will
remove all roots lying on the extension field but not in the base field.

For instance, for the system:
$$
\begin{align*}
&f_1 = 4x^3 + 2x^2y^2 + 12x - y - 5 \\
&f_2 = z (y^3 - 3x^2 - 7xy^3 - 1)(y - x) - 1 \\
\end{align*}
$$
the new polynomials would be:
$$
\begin{align*}
&f_3 = x^p - x\\
&f_4 = y^p - y\\
&f_5 = z^p - z
\end{align*}
$$
where $p$ is the prime of the field.

I am not sure, but maybe $f_5$ can be omitted, because maximum degree of $z$ is 1, so it should have no
values in the extension field.

The Gröbner basis of the extended set will contain a non-zero constant polynomial iff the system is
unsatisfiable. Such Gröbner basis can be calculated using any monomial ordering, like degree reverse
lexicographical order, which is cheaper to calculate than the lexicographical order.

I don't know how to turn this into a practical algorithm for the case where $p \gg 0$, as the procedure to
calculate the Gröbner basis is prohibitively expensive due to the time complexity being $O(p^2)$ on the
reduction step of $x^p - x = 0$. I suspect it can be done in $O(p \log p)$ by using exponentiation by
squaring, but even linear on $p$ is impractical, and wouldn't help with the huge size of each reduced
polynomial would have (on the order of $O(p)$). Maybe there can be a compact representation of the
polynomial, but still don't look promising...

## Local Search Method

This is more a complementary approach than an alternative, as it can not prove unsatisfiability, and
it may find a solution very quickly if they are abundant.

A local search is the standard way of approximating floating point solutions for pretty much every
equation system found in engineering and simulations. You treat the problem as a black box function
(called objective function), and variates the input (the solution you want) according to how it
affects the output, usually following the gradient towards a local minimum. If you get to zero, you
found a solution.

The traditional way of building an objective function for systems of equations is to take some norm
(usually $L_2$ or $L_\infty$) of the residual vector. In case of polynomials, the residual vector ends
up being the evaluation of each polynomial at the given input. In case of prime fields, it is important
to consider that the distance to 0 is at most $p/2$ when calculating the norm.

In SAT solving, discrete probabilistic methods are generally used for local search, and might be a good
choice in our case, too, like simulated annealing. But I think it is worth to investigate classical
mathematical methods, like Powell, BFGS or even Newton-Raphson, since the polynomials are differentiable.

That said, the problem is much harder on discrete case compared to continuous, because the search might
narrow down to oscillate around zero, but never reach because it is in between the minimum discrete
steps. Such problem does not happen over reals. A metaheuristic such as Taboo Search might be useful
to exclude such convergent points, but a huge number of restarts might be unavoidable.

Some ideas to consider: it might be beneficial to just use the equalities for the local search, removing
the inequalities polynomial and considering each individual inequality as a constraint. Or if the
inequalities happen to be useful in the search, it might be better to apply the Rabinowitsch trick
individually for each negation, because adding new variables is not a problem for this method, and makes
each restriction more explicit for the search algorithm.

## CDCL Approach

Paper [10] describes a solver for nonlinear constraints of polynomials over $\mathbb{Z}$ and its
roots over $\mathbb{R}$. It is done by a CDCL-style procedure where a sequence of clauses and variable
assignments is assembled one by one so that the total assignment is always consistent with the selected
clauses.

When a conflict is detected, the set of conflicting polynomial constraints is identified, and Cylindrical
Algebraic Decomposition (CAD) is used to isolate a whole region in the search space where this conflict can
happen. This region is then described as set of polynomial constraints that are added to the problem
as learned clauses.

The difficulty of generalizing this technique for prime fields is that we need a way for generalizing
a conflict in one particular variable evaluation to a whole region of the search space (the role of CAD
in the original algorithm), and also be able to describe it in a meaningful way for the deduction system
(i.e. probably define the new restrictions only in terms of $=$ and $\ne$, not with $>$, $\ge$, $<$ and
$\le$).

It also complicates the matter that we are only interested in solutions that happens to be on the base
field, not in the whole extension field, a restriction the original solver does not have to worry about.

Thesis [11] attempts two different approaches to replace CAD for the prime field case: in one,
a conflicting clause is found by calculating the Gröbner basis on the remaining set of 
polynomials after evaluation on the partial variable assignment. The other approach uses elimination
theory, which is able to handle equations of the kind $f \ne 0$ directly, where $f$ is a polynomial.

The problem of roots outside of the base field is solved by including the polynomials
$x^p - x$ for every variable $x$, which is most likely the responsible for the poor performance
displayed by the algorithms in the thesis, and limits the technique to very low values of $p$.

I have a gut feeling that it might be possible to improve [11] approach by removing the $x^p - x$
polynomials from the conflict search, and handle such restriction separately (as well as the common
restriction for binary variables $x^2 - x$). If possible, this technique is very promising.

## SIMPLEX generalization

In linear systems, the search can be reduced by looking at the bounds of the variables with the
SIMPLEX algorithm, so maybe there is a SIMPLEX generalization that works here to speed-up the search,
but I find it unlikely, as SIMPLEX works by iteratively restricting the convex search space limited
by straight lines/hyperplanes, but the curves designated by non-linear polynomials are not planes, so
the limited space will not be convex.

I just barely considered this possibility.

## Other alternatives

Thesis [9] looks very promising, I don't know how I missed it until now.

An algorithm for the problem named Mutant Zhuang-Zi is given in [8]. Looking very superficially,
doesn't seems to apply to large $p$ due to computations with $x^p - x$.

## References

[1]: [*Solving systems of polynomial congruences modulo a large prime* (1996), by Ming-Deh Huang and Yiu-Chung Wong](https://doi.org/10.1109/SFCS.1996.548470)

[2]: [*Fast Computation of a Rational Point of a Variety over a Finite Field* (2006), by Antonio Cafure and Guillermo Matera](https://www.jstor.org/stable/4100138)

[3]: [*Fast parallel absolute irreducibility testing* (1985), by Erich Kaltofen](https://www.sciencedirect.com/science/article/pii/S0747717185800298)

[4]: [*Solving Polynomial Systems over Finite Fields: Algorithms, Implementation and Applications* (2013) by Chenqi Mou](https://tel.archives-ouvertes.fr/tel-01110887)

[5]: [*A lower bound for the number of solutions of equations over finite fields* (1974) by Wolfgang M.Schmidt](https://www.sciencedirect.com/science/article/pii/0022314X74900432)

[6]: [*Gröbner bases and primary decomposition of polynomial ideals* (1988) by Patrizia Giannia, Barry Trager and Gail Zacharias](https://www.sciencedirect.com/science/article/pii/S0747717188800403)

[7]: [*Derandomizing some algebraic and number-theoretic algorithms* (2006) by Neeraj Kayal](https://www.microsoft.com/en-us/research/publication/derandomizing-some-algebraic-and-number-theoretic-algorithms/)

[8]: [*Mutant Zhuang-Zi Algorithm* (2010) by Jintai Ding and Dieter S. Schmidt](https://link.springer.com/chapter/10.1007/978-3-642-12929-2_3)

[9]: [*Counting Zeros over Finite Fields Using Gröbner Bases* (2009) by Sicun Gao](https://www.cs.cmu.edu/~sicung/papers/MS_thesis.pdf)

[10]: [*Solving Non-linear Arithmetic* (2012) by Dejan Jovanović and Leonardo de Moura](https://link.springer.com/chapter/10.1007/978-3-642-31365-3_27)

[11]: [*Non-Linear SMT-Reasoning over Finite Fields* (2022) by Thomas Hader](https://repositum.tuwien.at/handle/20.500.12708/19502)